In [1]:
# !pip install tensorflow==2.10.1
# !pip install tensorflow_probability==0.18.0
# !pip install transformers
import sys, os

# os.environ["CUDA_VISIBLE_DEVICES"] = ""
sys.path.append("../../bin")
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras
from eval_model_class import PredictionGTF
from models import lstm_model

2024-10-07 11:45:50.991994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-07 11:45:51.081760: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 11:45:51.516989: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/beckerf/mambaforge/envs/tiberiusdev/lib/:/home/beckerf/mambaforge/envs/tiberiusdev/lib/python3.10/site-packages/nvidia/cudnn/lib:
2024-10-07 11:45:51.517248: W tensorflow/stream_executor/platform/default/ds

In [4]:
batch_size = 2
seq_len = 500004
strand = '+'

emb = False
hmm_factor = 1
hmm_parallel = 817
trans_lstm = False

inp_data_dir = 'inp/'
out_dir = 'test_train/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

model_path = "../../model_weights/tiberius_weights"
genome_path = f'{inp_data_dir}/genome.fa'
# output gtf file
gtf_out = 'tiberius.py'

pred_gtf = PredictionGTF( 
    model_path=model_path,
     seq_len=seq_len, 
    batch_size=batch_size,
    hmm=True, 
    emb=False, 
    num_hmm=1,
    hmm_factor=1,
    genome_path=genome_path,
    softmask=True, strand=strand,
    parallel_factor=hmm_parallel
)

In [5]:
# load model
pred_gtf.load_model()

# load input data x_seq 
x_seq, y_seq, coords = pred_gtf.load_inp_data(    
    strand=strand, 
    chunk_coords=True, softmask=True
)

Running gene pred hmm layer with parallel factor 817
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, None, 6)]    0           []                               
                                                                                                  
 initial_conv (Conv1D)          (None, None, 128)    2432        ['main_input[0][0]']             
                                                                                                  
 layer_normalization1 (LayerNor  (None, None, 128)   256         ['initial_conv[0][0]']           
 malization)                                                                                      
                                                                                                  
 conv_1 (Conv1D)                (None, 

In [6]:
# generate LSTM and HMM predictions
hmm_pred = pred_gtf.get_predictions(x_seq, hmm_filter=True)

# infer gene structures and write GTF file
pred_gtf.create_gtf(y_label=hmm_pred, coords=coords,
        out_file=gtf_out, f_chunks=x_seq, strand=strand)

### LSTM prediction


2024-10-07 11:46:19.310002: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8800
2024-10-07 11:46:19.604464: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-07 11:46:19.604890: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-07 11:46:19.604902: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-10-07 11:46:19.605324: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-07 11:46:19.605362: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-10-07 11:46:19.762319: I tensorflow/stream_executor/cuda/c

LSTM took 2.0830203890800476 minutes to execute.
### HMM Viterbi
HMM took 0.3661714474360148 minutes to execute.
### LSTM prediction
LSTM took 0.2551252365112305 minutes to execute.
### HMM Viterbi
HMM took 0.03489230076471964 minutes to execute.


(<genome_anno.Anno at 0x7ef9f472e980>, 76)